In [4]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import talib

# Carregar o CSV com nome atualizado
df = pd.read_csv('BYBIT_BTCUSDT.P_1h.csv')

# Converter o timestamp para datetime (descomente se necessário)
if 'time' in df.columns:
    try:
        df['time'] = pd.to_datetime(df['time'], unit='s')
    except:
        pass  # Se o timestamp já estiver em formato datetime

# Atribuir os dados às variáveis
timestamp = df['time']
open_price = df['open']
high_price = df['high']
low_price = df['low']
close_price = df['close']

# Parâmetros configuráveis
emaShortLength = 11  # Período da EMA Curta
emaLongLength = 55  # Período da EMA Longa
rsiLength = 22  # Período do RSI
macdShort = 15  # Período Curto MACD
macdLong = 34  # Período Longo MACD
macdSignal = 11  # Período de Sinal MACD
adxLength = 16  # Período ADX (igual ao DI Length)
adxSmoothing = 13  # ADX Smoothing
adxThreshold = 12  # Nível de ADX para indicar tendência
bbLength = 14  # Período do Bollinger Bands
bbMultiplier = 1.7  # Multiplicador do Bollinger Bands
lateralThreshold = 0.005  # Limite de Lateralização
stopgain_lateral_long = 1.11
stoploss_lateral_long = 0.973
stopgain_lateral_short = 0.973
stoploss_lateral_short = 1.09
stopgain_normal_long = 1.32
stoploss_normal_long = 0.92
stopgain_normal_short = 0.77
stoploss_normal_short = 1.12

# Funções para cálculo manual do MACD
def macd_func(series, fast_period, slow_period, signal_period):
    ema_fast = talib.EMA(series, fast_period)
    ema_slow = talib.EMA(series, slow_period)
    macd_line = ema_fast - ema_slow
    signal_line = talib.EMA(macd_line, signal_period)
    macd_hist = macd_line - signal_line
    return macd_line, signal_line, macd_hist

def get_adx_manual(high, low, close, di_lookback, adx_smoothing):
    tr = []
    previous_close = close.iloc[0]
    plus_dm = []
    minus_dm = []

    for i in range(1, len(close)):
        current_plus_dm = high.iloc[i] - high.iloc[i-1]
        current_minus_dm = low.iloc[i-1] - low.iloc[i]
        plus_dm.append(max(current_plus_dm, 0) if current_plus_dm > current_minus_dm else 0)
        minus_dm.append(max(current_minus_dm, 0) if current_minus_dm > current_plus_dm else 0)

        tr1 = high.iloc[i] - low.iloc[i]
        tr2 = abs(high.iloc[i] - previous_close)
        tr3 = abs(low.iloc[i] - previous_close)
        true_range = max(tr1, tr2, tr3)
        tr.append(true_range)

        previous_close = close.iloc[i]

    tr.insert(0, np.nan)
    plus_dm.insert(0, np.nan)
    minus_dm.insert(0, np.nan)

    tr = pd.Series(tr)
    plus_dm = pd.Series(plus_dm)
    minus_dm = pd.Series(minus_dm)

    atr = tr.rolling(window=di_lookback).mean()

    plus_di = 100 * (plus_dm.ewm(alpha=1/di_lookback, adjust=False).mean() / atr)
    minus_di = 100 * (minus_dm.ewm(alpha=1/di_lookback, adjust=False).mean() / atr)

    dx = (abs(plus_di - minus_di) / (plus_di + minus_di)) * 100
    adx = dx.ewm(alpha=1/adx_smoothing, adjust=False).mean()

    return plus_di, minus_di, adx

# Cálculo dos indicadores
emaShort = talib.EMA(close_price, emaShortLength)
emaLong = talib.EMA(close_price, emaLongLength)
rsi = talib.RSI(close_price, timeperiod=rsiLength)
plus_di, minus_di, adx = get_adx_manual(high_price, low_price, close_price, adxLength, adxSmoothing)
adx = adx.fillna(0).astype(int)
macdLine, signalLine, macdHist = macd_func(close_price, macdShort, macdLong, macdSignal)
upperBand, middleBand, lowerBand = talib.BBANDS(close_price, timeperiod=bbLength, nbdevup=bbMultiplier, nbdevdn=bbMultiplier, matype=0)

# Função para detectar crossover (cruzamento ascendente)
def crossover(series1, series2):
    return (series1.shift(1) <= series2.shift(1)) & (series1 > series2)

# Função para detectar crossunder (cruzamento descendente)
def crossunder(series1, series2):
    return (series1.shift(1) >= series2.shift(1)) & (series1 < series2)

# Condição de mercado em tendência (baseada no ADX)
trendingMarket = adx >= adxThreshold

# Condições de lateralização com Bollinger Bands
bandWidth = (upperBand - lowerBand) / middleBand
isLateral = bandWidth < lateralThreshold

# Saldo inicial
saldo = 1_000_000  # Saldo inicial de 1.000.000
quantidade = 0  # Quantidade de BTC comprada/vendida na transação
position_open = False  # Indica se estamos em uma transação
current_position = None  # 'long' ou 'short'
entry_price = None  # Preço de entrada da posição
orders = []
trade_count = 0  # Contador de trades

# Condição Long
longCondition = (crossover(emaShort, emaLong)) & (rsi < 60) & (macdHist > 0.5) & trendingMarket

# Condição Short
shortCondition = (crossunder(emaShort, emaLong)) & (rsi > 40) & (macdHist < -0.5) & trendingMarket

commission = 0.00032  # 0,032%

# Loop para verificar e executar as ordens
for i in range(1, len(df)):
    adjusted_timestamp = timestamp.iloc[i]

    # Determinar a condição de lateralização da vela anterior
    lateral_prev = isLateral.iloc[i-1]

    # Estratégia de Mean Reversion para mercado lateral
    if lateral_prev:
        if not position_open:
            # Condição para entrar em Long
            if (close_price.iloc[i-1] < lowerBand.iloc[i-1]) and crossover(close_price, lowerBand).iloc[i-1] and longCondition.iloc[i-1]:
                entry_price = open_price.iloc[i]  # Executa no candle atual
                adjusted_timestamp_entry = timestamp.iloc[i]
                stopLossLong = entry_price * stoploss_lateral_long
                takeProfitLong = entry_price * stopgain_lateral_long
                quantidade = saldo / entry_price
                saldo -= saldo * commission
                orders.append(f"Entrar em transação (long lateral) em {adjusted_timestamp_entry} com preço {entry_price:.2f}, Stop Loss: {stopLossLong:.2f}, Take Profit: {takeProfitLong:.2f}")
                position_open = True
                current_position = 'long'
                trade_count += 1
                print(f"Trade LONG lateral executado em {adjusted_timestamp_entry} a {entry_price:.2f}")
            # Condição para entrar em Short
            elif (close_price.iloc[i-1] > upperBand.iloc[i-1]) and crossunder(close_price, upperBand).iloc[i-1] and shortCondition.iloc[i-1]:
                entry_price = open_price.iloc[i]
                adjusted_timestamp_entry = timestamp.iloc[i]
                stopLossShort = entry_price * stoploss_lateral_short
                takeProfitShort = entry_price * stopgain_lateral_short
                quantidade = saldo / entry_price
                saldo -= saldo * commission
                orders.append(f"Entrar em transação (short lateral) em {adjusted_timestamp_entry} com preço {entry_price:.2f}, Stop Loss: {stopLossShort:.2f}, Take Profit: {takeProfitShort:.2f}")
                position_open = True
                current_position = 'short'
                trade_count += 1
                print(f"Trade SHORT lateral executado em {adjusted_timestamp_entry} a {entry_price:.2f}")
        elif position_open:
            if current_position == 'long':
                # Condição para inverter para Short
                if (close_price.iloc[i] > upperBand.iloc[i]) and crossover(close_price, upperBand).iloc[i] and shortCondition.iloc[i-1]:
                    saldo = quantidade * close_price.iloc[i]
                    saldo -= saldo * commission
                    orders.append(f"Sair de transação (long) em {adjusted_timestamp} com preço {open_price.iloc[i]:.2f} (Inversão para Short), saldo atualizado: {saldo:.2f}")
                    entry_price = open_price.iloc[i]  # Executa no candle atual
                    adjusted_timestamp_entry = timestamp.iloc[i]
                    stopLossShort = entry_price * stoploss_lateral_short
                    takeProfitShort = entry_price * stopgain_lateral_short
                    quantidade = saldo / entry_price
                    saldo -= saldo * commission
                    orders.append(f"Entrar em transação (short lateral) em {adjusted_timestamp_entry} com preço {entry_price:.2f}, Stop Loss: {stopLossShort:.2f}, Take Profit: {takeProfitShort:.2f}")
                    current_position = 'short'
                    trade_count += 1
                    print(f"Trade SHORT lateral executado em {adjusted_timestamp_entry} a {entry_price:.2f}")
            elif current_position == 'short':
                # Condição para inverter para Long
                if (close_price.iloc[i] < lowerBand.iloc[i]) and crossover(close_price, lowerBand).iloc[i] and longCondition.iloc[i-1]:
                    saldo += quantidade * (entry_price - close_price.iloc[i])
                    saldo -= saldo * commission
                    orders.append(f"Sair de transação (short) em {adjusted_timestamp} com preço {open_price.iloc[i]:.2f} (Inversão para Long), saldo atualizado: {saldo:.2f}")
                    entry_price = open_price.iloc[i]
                    adjusted_timestamp_entry = timestamp.iloc[i]
                    stopLossLong = entry_price * stoploss_lateral_long
                    takeProfitLong = entry_price * stopgain_lateral_long
                    quantidade = saldo / entry_price
                    saldo -= saldo * commission
                    orders.append(f"Entrar em transação (long lateral) em {adjusted_timestamp_entry} com preço {entry_price:.2f}, Stop Loss: {stopLossLong:.2f}, Take Profit: {takeProfitLong:.2f}")
                    current_position = 'long'
                    trade_count += 1
                    print(f"Trade LONG lateral executado em {adjusted_timestamp_entry} a {entry_price:.2f}")

            # Gerenciamento de posições abertas com base nos parâmetros laterais da vela anterior
            if current_position == 'long':
                stopLossLong = entry_price * stoploss_lateral_long
                takeProfitLong = entry_price * stopgain_lateral_long
                if low_price.iloc[i] <= stopLossLong:
                    saldo = quantidade * stopLossLong
                    orders.append(f"Sair de transação (long) em {adjusted_timestamp} com preço {stopLossLong:.2f} (Stoploss Lateral), saldo atualizado: {saldo:.2f}")
                    position_open = False
                    print(f"Stop Loss LONG atingido em {adjusted_timestamp} a {stopLossLong:.2f}")
                elif high_price.iloc[i] >= takeProfitLong:
                    saldo = quantidade * takeProfitLong
                    orders.append(f"Sair de transação (long) em {adjusted_timestamp} com preço {takeProfitLong:.2f} (Take Profit Lateral), saldo atualizado: {saldo:.2f}")
                    position_open = False
                    print(f"Take Profit LONG atingido em {adjusted_timestamp} a {takeProfitLong:.2f}")
            elif current_position == 'short':
                stopLossShort = entry_price * stoploss_lateral_short
                takeProfitShort = entry_price * stopgain_lateral_short  # Corrigido para takeProfitShort
                if high_price.iloc[i] >= stopLossShort:
                    saldo -= quantidade * (stopLossShort - entry_price)
                    orders.append(f"Sair de transação (short) em {adjusted_timestamp} com preço {stopLossShort:.2f} (Stoploss Lateral), saldo atualizado: {saldo:.2f}")
                    position_open = False
                    print(f"Stop Loss SHORT atingido em {adjusted_timestamp} a {close_price.iloc[i]:.2f}")
                elif low_price.iloc[i] <= takeProfitShort:
                    saldo += quantidade * (entry_price - takeProfitShort)
                    orders.append(f"Sair de transação (short) em {adjusted_timestamp} com preço {takeProfitShort:.2f} (Take Profit Lateral), saldo atualizado: {saldo:.2f}")
                    position_open = False
                    print(f"Take Profit SHORT atingido em {adjusted_timestamp} a {takeProfitShort:.2f}")

    # Estratégia para mercado não lateral
    if not lateral_prev:
        if not position_open:
            # Condição para entrar em Long
            if longCondition.iloc[i-1]:
                entry_price = open_price.iloc[i]
                adjusted_timestamp_entry = timestamp.iloc[i]
                stopLossLong = entry_price * stoploss_normal_long
                takeProfitLong = entry_price * stopgain_normal_long
                quantidade = saldo / entry_price
                saldo -= saldo * commission
                orders.append(f"Entrar em transação (long normal) em {adjusted_timestamp_entry} com preço {entry_price:.2f}, Stop Loss: {stopLossLong:.2f}, Take Profit: {takeProfitLong:.2f}")
                position_open = True
                current_position = 'long'
                trade_count += 1
                print(f"Trade LONG normal executado em {adjusted_timestamp_entry} a {entry_price:.2f}")
            # Condição para entrar em Short
            elif shortCondition.iloc[i-1]:
                entry_price = open_price.iloc[i]
                adjusted_timestamp_entry = timestamp.iloc[i]
                stopLossShort = entry_price * stoploss_normal_short
                takeProfitShort = entry_price * stopgain_normal_short
                quantidade = saldo / entry_price
                saldo -= saldo * commission
                orders.append(f"Entrar em transação (short normal) em {adjusted_timestamp_entry} com preço {entry_price:.2f}, Stop Loss: {stopLossShort:.2f}, Take Profit: {takeProfitShort:.2f}")
                position_open = True
                current_position = 'short'
                trade_count += 1
                print(f"Trade SHORT normal executado em {adjusted_timestamp_entry} a {entry_price:.2f}")
        elif position_open:
            if current_position == 'long':
                # Condição para inverter para Short
                if shortCondition.iloc[i-1]:
                    saldo = quantidade * open_price.iloc[i]
                    saldo -= saldo * commission
                    orders.append(f"Sair de transação (long) em {adjusted_timestamp} com preço {open_price.iloc[i]:.2f} (Inversão para Short), saldo atualizado: {saldo:.2f}")
                    entry_price = open_price.iloc[i]
                    adjusted_timestamp_entry = timestamp.iloc[i]
                    stopLossShort = entry_price * stoploss_normal_short
                    takeProfitShort = entry_price * stopgain_normal_short
                    quantidade = saldo / entry_price
                    saldo -= saldo * commission
                    orders.append(f"Entrar em transação (short normal) em {adjusted_timestamp_entry} com preço {entry_price:.2f}, Stop Loss: {stopLossShort:.2f}, Take Profit: {takeProfitShort:.2f}")
                    current_position = 'short'
                    trade_count += 1
                    print(f"Trade SHORT normal executado em {adjusted_timestamp_entry} a {entry_price:.2f}")
            elif current_position == 'short':
                # Condição para inverter para Long
                if longCondition.iloc[i-1]:
                    saldo += quantidade * (entry_price - open_price.iloc[i])
                    saldo -= saldo * commission
                    orders.append(f"Sair de transação (short) em {adjusted_timestamp} com preço {open_price.iloc[i]:.2f} (Inversão para Long), saldo atualizado: {saldo:.2f}")
                    entry_price = open_price.iloc[i]
                    adjusted_timestamp_entry = timestamp.iloc[i]
                    stopLossLong = entry_price * stoploss_normal_long
                    takeProfitLong = entry_price * stopgain_normal_long
                    quantidade = saldo / entry_price
                    saldo -= saldo * commission
                    orders.append(f"Entrar em transação (long normal) em {adjusted_timestamp_entry} com preço {entry_price:.2f}, Stop Loss: {stopLossLong:.2f}, Take Profit: {takeProfitLong:.2f}")
                    current_position = 'long'
                    trade_count += 1
                    print(f"Trade LONG normal executado em {adjusted_timestamp_entry} a {entry_price:.2f}")

            # Gerenciamento de posições abertas com base nos parâmetros não laterais
            if current_position == 'long':
                stopLossLong = entry_price * stoploss_normal_long
                takeProfitLong = entry_price * stopgain_normal_long
                if low_price.iloc[i] <= stopLossLong:
                    saldo = quantidade * stopLossLong
                    orders.append(f"Sair de transação (long) em {adjusted_timestamp} com preço {stopLossLong:.2f} (Stoploss), saldo atualizado: {saldo:.2f}")
                    position_open = False
                    print(f"Stop Loss LONG normal atingido em {adjusted_timestamp} a {stopLossLong:.2f}")
                elif high_price.iloc[i] >= takeProfitLong:
                    saldo = quantidade * takeProfitLong
                    orders.append(f"Sair de transação (long) em {adjusted_timestamp} com preço {takeProfitLong:.2f} (Take Profit), saldo atualizado: {saldo:.2f}")
                    position_open = False
                    print(f"Take Profit LONG normal atingido em {adjusted_timestamp} a {takeProfitLong:.2f}")
            elif current_position == 'short':
                stopLossShort = entry_price * stoploss_normal_short
                takeProfitShort = entry_price * stopgain_normal_short  # Corrigido para takeProfitShort
                if high_price.iloc[i] >= stopLossShort:
                    saldo -= quantidade * (stopLossShort - entry_price)
                    orders.append(f"Sair de transação (short) em {adjusted_timestamp} com preço {open_price.iloc[i]:.2f} (Stoploss), saldo atualizado: {saldo:.2f}")
                    position_open = False
                    print(f"Stop Loss SHORT normal atingido em {adjusted_timestamp} a {stopLossShort:.2f}")
                elif low_price.iloc[i] <= takeProfitShort:
                    saldo += quantidade * (entry_price - takeProfitShort)
                    orders.append(f"Sair de transação (short) em {adjusted_timestamp} com preço {takeProfitShort:.2f} (Take Profit), saldo atualizado: {saldo:.2f}")
                    position_open = False
                    print(f"Take Profit SHORT normal atingido em {adjusted_timestamp} a {takeProfitShort:.2f}")

# Exibir as ordens geradas
for order in orders:
    print(order)

# Exibir o número total de trades
print(f"Total de trades: {trade_count}")


Trade SHORT normal executado em 2020-04-01 07:00:00 a 6289.50
Stop Loss SHORT normal atingido em 2020-04-02 17:00:00 a 7044.24
Trade SHORT normal executado em 2020-04-10 04:00:00 a 7175.50
Trade LONG normal executado em 2020-04-14 06:00:00 a 6881.00
Take Profit LONG normal atingido em 2020-04-30 03:00:00 a 9082.92
Trade SHORT normal executado em 2020-05-04 01:00:00 a 8806.00
Trade LONG normal executado em 2020-05-04 22:00:00 a 8938.00
Trade SHORT normal executado em 2020-05-09 23:00:00 a 9604.50
Trade LONG normal executado em 2020-05-12 18:00:00 a 8872.50
Trade SHORT normal executado em 2020-05-16 00:00:00 a 9315.00
Trade LONG normal executado em 2020-05-16 13:00:00 a 9466.50
Trade SHORT normal executado em 2020-05-19 05:00:00 a 9535.50
Trade LONG normal executado em 2020-05-24 09:00:00 a 9263.00
Trade SHORT normal executado em 2020-06-01 00:00:00 a 9449.50
Trade LONG normal executado em 2020-06-04 05:00:00 a 9644.00
Trade SHORT normal executado em 2020-06-05 23:00:00 a 9677.00
Trade L

In [3]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import talib

# Load the CSV with updated name
df = pd.read_csv('BYBIT_BTCUSDT.P_1h.csv')
# df['time'] = pd.to_datetime(df['time'], unit='s')

# Assign data to variables
timestamp = df['time']
open_price = df['open']
high_price = df['high']
low_price = df['low']
close_price = df['close']
# volume = df['Volume']

# Configurable parameters
emaShortLength = 11  # Short EMA period
emaLongLength = 55  # Long EMA period
rsiLength = 22  # RSI period
macdShort = 15  # MACD short period
macdLong = 34  # MACD long period
macdSignal = 11  # MACD signal period
adxLength = 16  # ADX period (equal to DI Length)
adxSmoothing = 13  # ADX Smoothing
adxThreshold = 12  # ADX level to indicate trend
bbLength = 14  # Bollinger Bands period
bbMultiplier = 1.7  # Bollinger Bands multiplier
lateralThreshold = 0.005  # Lateralization limit
stopgain_lateral_long = 1.11
stoploss_lateral_long = 0.973
stopgain_lateral_short = 0.973
stoploss_lateral_short = 1.09
stopgain_normal_long = 1.32
stoploss_normal_long = 0.92
stopgain_normal_short = 0.77
stoploss_normal_short = 1.12

# Functions for manual MACD calculation
def macd(series, fast_period, slow_period, signal_period):
    ema_fast = talib.EMA(series, fast_period)
    ema_slow = talib.EMA(series, slow_period)
    macd_line = ema_fast - ema_slow
    signal_line = talib.EMA(macd_line, signal_period)
    macd_hist = macd_line - signal_line
    return macd_line, signal_line, macd_hist

def get_adx_manual(high, low, close, di_lookback, adx_smoothing):
    # Initialize lists to store results
    tr = []
    previous_close = close[0]  # First closing value
    
    plus_dm = []
    minus_dm = []

    for i in range(1, len(close)):
        # Calculate +DM and -DM
        current_plus_dm = high[i] - high[i-1]
        current_minus_dm = low[i-1] - low[i]
        plus_dm.append(max(current_plus_dm, 0) if current_plus_dm > current_minus_dm else 0)
        minus_dm.append(max(current_minus_dm, 0) if current_minus_dm > current_plus_dm else 0)
        
        # Calculate True Range (TR)
        tr1 = high[i] - low[i]
        tr2 = abs(high[i] - previous_close)
        tr3 = abs(low[i] - previous_close)
        true_range = max(tr1, tr2, tr3)
        tr.append(true_range)
        
        # Update previous close
        previous_close = close[i]
    
    # Add a NaN value to the first position since there's no previous close for the first candle
    tr.insert(0, None)
    plus_dm.insert(0, None)
    minus_dm.insert(0, None)
    
    # Convert lists to pandas Series
    tr = pd.Series(tr)
    plus_dm = pd.Series(plus_dm)
    minus_dm = pd.Series(minus_dm)
    
    # Calculate ATR using the provided DI lookback
    atr = tr.rolling(window=di_lookback).mean()
    
    # Calculate +DI and -DI with DI lookback
    plus_di = 100 * (plus_dm.ewm(alpha=1/di_lookback).mean() / atr)
    minus_di = 100 * (minus_dm.ewm(alpha=1/di_lookback).mean() / atr)
    
    # Calculate DX
    dx = (abs(plus_di - minus_di) / (plus_di + minus_di)) * 100
    
    # Calculate ADX using specific smoothing
    adx = dx.ewm(alpha=1/adx_smoothing).mean()
    
    # Return calculated values
    return plus_di, minus_di, adx

# Calculate indicators
emaShort = np.ceil(talib.EMA(close_price, emaShortLength))
emaLong = np.ceil(talib.EMA(close_price, emaLongLength))

rsi = talib.RSI(close_price, timeperiod=rsiLength)
plus_di, minus_di, adx = get_adx_manual(high_price, low_price, close_price, adxLength, adxSmoothing)
adx = adx.fillna(0).astype(int)
macdLine, signalLine, macdHist = macd(close_price, macdShort, macdLong, macdSignal)
upperBand, middleBand, lowerBand = talib.BBANDS(close_price, timeperiod=bbLength, nbdevup=bbMultiplier, nbdevdn=bbMultiplier, matype=0)

# Function to detect crossover (upward crossing)
def crossover(series1, series2):
    series1_vals = series1.values
    series2_vals = series2.values
    cross = (series1_vals[1:] > series2_vals[1:]) & (series1_vals[:-1] <= series2_vals[:-1])
    cross = np.concatenate(([False], cross))  # Prepend False to align lengths
    return pd.Series(cross, index=series1.index)

# Function to detect crossunder (downward crossing)
def crossunder(series1, series2):
    series1_vals = series1.values
    series2_vals = series2.values
    cross = (series1_vals[1:] < series2_vals[1:]) & (series1_vals[:-1] >= series2_vals[:-1])
    cross = np.concatenate(([False], cross))  # Prepend False to align lengths
    return pd.Series(cross, index=series1.index)

# Market trending condition (based on ADX)
trendingMarket = adx >= adxThreshold

# Lateralization conditions with Bollinger Bands
bandWidth = (upperBand - lowerBand) / middleBand

isLateral = bandWidth < lateralThreshold

# Define EMA crossover and crossunder states
ema_crossover_state = emaShort > emaLong
ema_crossunder_state = emaShort < emaLong

# Long and Short Conditions
longCondition = ema_crossover_state & (rsi < 60) & (macdHist > 0.5) & trendingMarket
shortCondition = ema_crossunder_state & (rsi > 40) & (macdHist < -0.5) & trendingMarket

# Initial balance and trading variables
saldo = 1_000_000  # Initial balance of 1,000,000
quantidade = 0  # Quantity of BTC bought/sold in the transaction
position_open = False  # Indicates if we are in a transaction
current_position = None  # 'long' or 'short'
entry_price = None  # Entry price of the position
orders = []
trade_count = 0  # Trade counter
commission = 0.00032  # 0.032%

# Loop to check and execute orders
for i in range(len(df)):
    adjusted_timestamp = timestamp[i]
    
    # Determine the lateralization condition of the previous candle
    if i == 0:
        lateral_prev = False  # No previous candle for the first candle
    else:
        lateral_prev = isLateral[i-1]
    
    if lateral_prev:
        if not position_open:
            if (close_price[i] < lowerBand[i]) and crossover(close_price, lowerBand)[i] and longCondition[i]:
                entry_price = close_price[i]
                stopLossLong = entry_price * stoploss_lateral_long
                takeProfitLong = entry_price * stopgain_lateral_long
                quantidade = saldo / entry_price
                # Apply commission on buy
                saldo -= saldo * commission
                orders.append(f"enter long position (lateral) at {adjusted_timestamp} with price {entry_price}, Stop Loss: {stopLossLong}, Take Profit: {takeProfitLong}")
                position_open = True
                current_position = 'long'
                trade_count += 1
            elif (close_price[i] > upperBand[i]) and crossover(close_price, upperBand)[i] and shortCondition[i]:
                entry_price = close_price[i]
                stopLossShort = entry_price * stoploss_lateral_short
                takeProfitShort = entry_price * stopgain_lateral_short
                quantidade = saldo / entry_price
                # Apply commission on buy
                saldo -= saldo * commission
                orders.append(f"enter short position (lateral) at {adjusted_timestamp} with price {entry_price}, Stop Loss: {stopLossShort}, Take Profit: {takeProfitShort}")
                position_open = True
                current_position = 'short'
                trade_count += 1
        elif position_open:
            if current_position == 'long':
                if (close_price[i] > upperBand[i]) and crossover(close_price, upperBand)[i]:
                    if shortCondition[i]:
                        saldo = quantidade * close_price[i]
                        # Apply commission on sell
                        saldo -= saldo * commission
                        orders.append(f"exit long position at {adjusted_timestamp} with price {close_price[i]} (Reverse to Short), updated balance: {saldo:.2f}")
                        entry_price = close_price[i]
                        stopLossShort = entry_price * stoploss_lateral_short
                        takeProfitShort = entry_price * stopgain_lateral_short
                        quantidade = saldo / entry_price
                        # Apply commission on buy
                        saldo -= saldo * commission
                        orders.append(f"enter short position (lateral) at {adjusted_timestamp} with price {entry_price}, Stop Loss: {stopLossShort}, Take Profit: {takeProfitShort}")
                        current_position = 'short'
                        trade_count += 1
            elif current_position == 'short':
                if (close_price[i] < lowerBand[i]) and crossover(close_price, lowerBand)[i]:
                    if longCondition[i]:
                        saldo = saldo + (quantidade * (entry_price - close_price[i]))
                        # Apply commission on sell
                        saldo -= saldo * commission
                        orders.append(f"exit short position at {adjusted_timestamp} with price {close_price[i]} (Reverse to Long), updated balance: {saldo:.2f}")
                        entry_price = close_price[i]
                        stopLossLong = entry_price * stoploss_lateral_long
                        takeProfitLong = entry_price * stopgain_lateral_long
                        quantidade = saldo / entry_price
                        # Apply commission on buy
                        saldo -= saldo * commission
                        orders.append(f"enter long position (lateral) at {adjusted_timestamp} with price {entry_price}, Stop Loss: {stopLossLong}, Take Profit: {takeProfitLong}")
                        current_position = 'long'
                        trade_count += 1
            if current_position == 'long':
                stopLossLong = entry_price * stoploss_lateral_long
                takeProfitLong = entry_price * stopgain_lateral_long
                if low_price[i] <= stopLossLong:
                    saldo = quantidade * stopLossLong
                    orders.append(f"exit long position at {adjusted_timestamp} with price {stopLossLong} (Stoploss Lateral), updated balance: {saldo:.2f}")
                    position_open = False
                elif high_price[i] >= takeProfitLong:
                    saldo = quantidade * takeProfitLong
                    orders.append(f"exit long position at {adjusted_timestamp} with price {takeProfitLong} (Take Profit Lateral), updated balance: {saldo:.2f}")
                    position_open = False
            elif current_position == 'short':
                stopLossShort = entry_price * stoploss_lateral_short
                takeProfitShort = entry_price * stopgain_lateral_short
                if high_price[i] >= stopLossShort:
                    saldo = saldo - (quantidade * (stopLossShort - entry_price))
                    orders.append(f"exit short position at {adjusted_timestamp} with price {close_price[i]} (Stoploss Lateral), updated balance: {saldo:.2f}")
                    position_open = False
                elif low_price[i] <= takeProfitShort:
                    saldo = saldo + (quantidade * (entry_price - takeProfitShort))
                    orders.append(f"exit short position at {adjusted_timestamp} with price {close_price[i]} (Take Profit Lateral), updated balance: {saldo:.2f}")
                    position_open = False    
    else:  # Not lateral
        if not position_open:
            if longCondition[i]:
                entry_price = close_price[i]
                stopLossLong = entry_price * stoploss_normal_long
                takeProfitLong = entry_price * stopgain_normal_long
                quantidade = saldo / entry_price
                orders.append(f"enter long position (normal) at {adjusted_timestamp} with price {entry_price}, Stop Loss: {stopLossLong}, Take Profit: {takeProfitLong}")
                position_open = True
                current_position = 'long'
                trade_count += 1
            elif shortCondition[i]:
                entry_price = close_price[i]
                stopLossShort = entry_price * stoploss_normal_short
                takeProfitShort = entry_price * stopgain_normal_short
                quantidade = saldo / entry_price
                orders.append(f"enter short position (normal) at {adjusted_timestamp} with price {entry_price}, Stop Loss: {stopLossShort}, Take Profit: {takeProfitShort}")
                position_open = True
                current_position = 'short'
                trade_count += 1
        elif position_open:
            if current_position == 'long':
                if shortCondition[i]:
                    saldo = quantidade * close_price[i]
                    orders.append(f"exit long position at {adjusted_timestamp} with price {close_price[i]} (Reverse to Short), updated balance: {saldo:.2f}")
                    entry_price = close_price[i]
                    stopLossShort = entry_price * stoploss_normal_short
                    takeProfitShort = entry_price * stopgain_normal_short
                    quantidade = saldo / entry_price
                    orders.append(f"enter short position (normal) at {adjusted_timestamp} with price {entry_price}, Stop Loss: {stopLossShort}, Take Profit: {takeProfitShort}")
                    current_position = 'short'
                    trade_count += 1
            elif current_position == 'short':
                if longCondition[i]:
                    saldo = saldo + (quantidade * (entry_price - close_price[i]))
                    orders.append(f"exit short position at {adjusted_timestamp} with price {close_price[i]} (Reverse to Long), updated balance: {saldo:.2f}")
                    entry_price = close_price[i]
                    stopLossLong = entry_price * stoploss_normal_long
                    takeProfitLong = entry_price * stopgain_normal_long
                    quantidade = saldo / entry_price
                    orders.append(f"enter long position (normal) at {adjusted_timestamp} with price {entry_price}, Stop Loss: {stopLossLong}, Take Profit: {takeProfitLong}")
                    current_position = 'long'
                    trade_count += 1
            if current_position == 'long':
                stopLossLong = entry_price * stoploss_normal_long
                takeProfitLong = entry_price * stopgain_normal_long
                if low_price[i] <= stopLossLong:
                    saldo = quantidade * stopLossLong
                    orders.append(f"exit long position at {adjusted_timestamp} with price {stopLossLong} (Stoploss), updated balance: {saldo:.2f}")
                    position_open = False
                elif high_price[i] >= takeProfitLong:
                    saldo = quantidade * takeProfitLong
                    orders.append(f"exit long position at {adjusted_timestamp} with price {takeProfitLong} (Take Profit), updated balance: {saldo:.2f}")
                    position_open = False
            elif current_position == 'short':
                stopLossShort = entry_price * stoploss_normal_short
                takeProfitShort = entry_price * stopgain_normal_short
                if high_price[i] >= stopLossShort:
                    saldo = saldo - (quantidade * (stopLossShort - entry_price))
                    orders.append(f"exit short position at {adjusted_timestamp} with price {stopLossShort} (Stoploss), updated balance: {saldo:.2f}")
                    position_open = False
                elif low_price[i] <= takeProfitShort:
                    saldo = saldo + (quantidade * (entry_price - takeProfitShort))
                    orders.append(f"exit short position at {adjusted_timestamp} with price {takeProfitShort} (Take Profit), updated balance: {saldo:.2f}")
                    position_open = False

# Display the generated orders
for order in orders:
    print(order)

# Display the total number of trades
print(f"Total trades: {trade_count}")


enter short position (normal) at 2020-03-27 16:00:00 with price 6638.5, Stop Loss: 7435.120000000001, Take Profit: 5111.645
exit short position at 2020-03-30 12:00:00 with price 6302.5 (Reverse to Long), updated balance: 1050613.84
enter long position (normal) at 2020-03-30 12:00:00 with price 6302.5, Stop Loss: 5798.3, Take Profit: 8319.300000000001
exit long position at 2020-04-01 06:00:00 with price 6289.5 (Reverse to Short), updated balance: 1048446.77
enter short position (normal) at 2020-04-01 06:00:00 with price 6289.5, Stop Loss: 7044.240000000001, Take Profit: 4842.915
exit short position at 2020-04-02 17:00:00 with price 7044.240000000001 (Stoploss), updated balance: 922633.16
enter long position (normal) at 2020-04-02 18:00:00 with price 6787.0, Stop Loss: 6244.04, Take Profit: 8958.84
exit long position at 2020-04-10 03:00:00 with price 7175.5 (Reverse to Short), updated balance: 975446.33
enter short position (normal) at 2020-04-10 03:00:00 with price 7175.5, Stop Loss: 80

In [10]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import talib
from itertools import product
from tqdm import tqdm  # Para mostrar uma barra de progresso
import random

# Função para calcular o MACD manualmente
def macd(series, fast_period, slow_period, signal_period):
    ema_fast = talib.EMA(series, fast_period)
    ema_slow = talib.EMA(series, slow_period)
    macd_line = ema_fast - ema_slow
    signal_line = talib.EMA(macd_line, signal_period)
    macd_hist = macd_line - signal_line
    return macd_line, signal_line, macd_hist

# Função para calcular o ADX manualmente
def get_adx_manual(high, low, close, di_lookback, adx_smoothing):
    tr = []
    previous_close = close[0]  # Primeiro valor de fechamento
    
    plus_dm = []
    minus_dm = []

    for i in range(1, len(close)):
        # Calcular +DM e -DM
        current_plus_dm = high[i] - high[i-1]
        current_minus_dm = low[i-1] - low[i]
        plus_dm.append(max(current_plus_dm, 0) if current_plus_dm > current_minus_dm else 0)
        minus_dm.append(max(current_minus_dm, 0) if current_minus_dm > current_plus_dm else 0)
        
        # Calcular True Range (TR)
        tr1 = high[i] - low[i]
        tr2 = abs(high[i] - previous_close)
        tr3 = abs(low[i] - previous_close)
        true_range = max(tr1, tr2, tr3)
        tr.append(true_range)
        
        # Atualizar o fechamento anterior
        previous_close = close[i]
    
    # Adiciona um valor NaN para a primeira posição
    tr.insert(0, None)
    plus_dm.insert(0, None)
    minus_dm.insert(0, None)
    
    # Converter listas para Series do pandas
    tr = pd.Series(tr)
    plus_dm = pd.Series(plus_dm)
    minus_dm = pd.Series(minus_dm)
    
    # Calcular o ATR usando o lookback fornecido para DI
    atr = tr.rolling(window=di_lookback).mean()
    
    # Calcular DI+ e DI- com o lookback de DI
    plus_di = 100 * (plus_dm.ewm(alpha=1/di_lookback).mean() / atr)
    minus_di = 100 * (minus_dm.ewm(alpha=1/di_lookback).mean() / atr)
    
    # Calcular o DX
    dx = (abs(plus_di - minus_di) / (plus_di + minus_di)) * 100
    
    # Calcular o ADX usando o smoothing específico para o ADX
    adx = dx.ewm(alpha=1/adx_smoothing).mean()
    
    return plus_di, minus_di, adx

# Função para detectar crossover (cruzamento ascendente)
def crossover(series1, series2):
    series1_vals = series1.values
    series2_vals = series2.values
    cross = (series1_vals[1:] > series2_vals[1:]) & (series1_vals[:-1] <= series2_vals[:-1])
    cross = np.concatenate(([False], cross))  # Prepend False para alinhar os comprimentos
    return pd.Series(cross, index=series1.index)

# Função para detectar crossunder (cruzamento descendente)
def crossunder(series1, series2):
    series1_vals = series1.values
    series2_vals = series2.values
    cross = (series1_vals[1:] < series2_vals[1:]) & (series1_vals[:-1] >= series2_vals[:-1])
    cross = np.concatenate(([False], cross))  # Prepend False para alinhar os comprimentos
    return pd.Series(cross, index=series1.index)

# Função de Backtest
def backtest(params, df):
    # Desempacotar parâmetros
    (emaShortLength, emaLongLength, rsiLength, macdShort, macdLong, macdSignal,
     adxLength, adxSmoothing, adxThreshold, bbLength, bbMultiplier,
     lateralThreshold, stopgain_lateral_long, stoploss_lateral_long,
     stopgain_lateral_short, stoploss_lateral_short,
     stopgain_normal_long, stoploss_normal_long,
     stopgain_normal_short, stoploss_normal_short,
     commission) = params
    
    # Atribuir os dados às variáveis
    timestamp = df['time']
    open_price = df['open']
    high_price = df['high']
    low_price = df['low']
    close_price = df['close']
    
    # Cálculo dos Indicadores
    emaShort = talib.EMA(close_price, emaShortLength).round(0)
    emaLong = talib.EMA(close_price, emaLongLength).round(0)
    rsi = talib.RSI(close_price, timeperiod=rsiLength)
    plus_di, minus_di, adx = get_adx_manual(high_price, low_price, close_price, adxLength, adxSmoothing)
    adx = adx.fillna(0).astype(int)
    macdLine, signalLine, macdHist = macd(close_price, macdShort, macdLong, macdSignal)
    upperBand, middleBand, lowerBand = talib.BBANDS(close_price, timeperiod=bbLength, nbdevup=bbMultiplier, nbdevdn=bbMultiplier, matype=0)
    
    # Condição de mercado em tendência (baseada no ADX)
    trendingMarket = adx >= adxThreshold
    
    # Condições de lateralização com Bollinger Bands
    bandWidth = (upperBand - lowerBand) / middleBand
    isLateral = bandWidth < lateralThreshold
    
    # Condições de crossover
    longCondition = (crossover(emaShort, emaLong)) & (rsi < 60) & (macdHist > 0.5) & trendingMarket
    shortCondition = (crossunder(emaShort, emaLong)) & (rsi > 40) & (macdHist < -0.5) & trendingMarket
    
    # Inicialização do Backtest
    saldo = 1_000_000  # Saldo inicial de 1.000.000
    quantidade = 0  # Quantidade de BTC comprada/vendida na transação
    position_open = False  # Indica se estamos em uma transação
    current_position = None  # 'long' ou 'short'
    entry_price = None  # Preço de entrada da posição
    trade_count = 0  # Contador de trades
    
    # Loop para verificar e executar as ordens
    for i in range(len(df)):
        # Estratégia de Mean Reversion para mercado lateral
        if isLateral[i]:
            if not position_open:
                if (close_price[i] < lowerBand[i]) and crossover(close_price, lowerBand)[i] and longCondition[i]:
                    entry_price = close_price[i]
                    stopLossLong = entry_price * stoploss_lateral_long
                    takeProfitLong = entry_price * stopgain_lateral_long
                    quantidade = saldo / entry_price
                    # Aplicar taxa de corretora na compra
                    saldo -= saldo * commission
                    position_open = True
                    current_position = 'long'
                    trade_count += 1
                elif (close_price[i] > upperBand[i]) and crossover(close_price, upperBand)[i] and shortCondition[i]:
                    entry_price = close_price[i]
                    stopLossShort = entry_price * stoploss_lateral_short
                    takeProfitShort = entry_price * stopgain_lateral_short
                    quantidade = saldo / entry_price
                    # Aplicar taxa de corretora na venda
                    saldo -= saldo * commission
                    position_open = True
                    current_position = 'short'
                    trade_count += 1
            elif position_open:
                if current_position == 'long':
                    if (close_price[i] > upperBand[i]) and crossover(close_price, upperBand)[i] and shortCondition[i]:
                        saldo = quantidade * close_price[i]
                        # Aplicar taxa de corretora na venda
                        saldo -= saldo * commission
                        # Inversão para Short
                        entry_price = close_price[i]
                        stopLossShort = entry_price * stoploss_lateral_short
                        takeProfitShort = entry_price * stopgain_lateral_short
                        quantidade = saldo / entry_price
                        # Aplicar taxa de corretora na compra
                        saldo -= saldo * commission
                        current_position = 'short'
                        trade_count += 1
                elif current_position == 'short':
                    if (close_price[i] < lowerBand[i]) and crossover(close_price, lowerBand)[i] and longCondition[i]:
                        saldo = saldo + (quantidade * (entry_price - close_price[i]))
                        # Aplicar taxa de corretora na venda
                        saldo -= saldo * commission
                        # Inversão para Long
                        entry_price = close_price[i]
                        stopLossLong = entry_price * stoploss_lateral_long
                        takeProfitLong = entry_price * stopgain_lateral_long
                        quantidade = saldo / entry_price
                        # Aplicar taxa de corretora na compra
                        saldo -= saldo * commission
                        current_position = 'long'
                        trade_count += 1

        # Estratégia para mercado não lateral
        if not isLateral[i]:
            if not position_open:
                if longCondition[i]:
                    entry_price = close_price[i]
                    stopLossLong = entry_price * stoploss_normal_long
                    takeProfitLong = entry_price * stopgain_normal_long
                    quantidade = saldo / entry_price
                    # Aplicar taxa de corretora na compra
                    saldo -= saldo * commission
                    position_open = True
                    current_position = 'long'
                    trade_count += 1
                elif shortCondition[i]:
                    entry_price = close_price[i]
                    stopLossShort = entry_price * stoploss_normal_short
                    takeProfitShort = entry_price * stopgain_normal_short
                    quantidade = saldo / entry_price
                    # Aplicar taxa de corretora na venda
                    saldo -= saldo * commission
                    position_open = True
                    current_position = 'short'
                    trade_count += 1
            elif position_open:
                if current_position == 'long':
                    if shortCondition[i]:
                        saldo = quantidade * close_price[i]
                        # Aplicar taxa de corretora na venda
                        saldo -= saldo * commission
                        # Inversão para Short
                        entry_price = close_price[i]
                        stopLossShort = entry_price * stoploss_normal_short
                        takeProfitShort = entry_price * stopgain_normal_short
                        quantidade = saldo / entry_price
                        # Aplicar taxa de corretora na compra
                        saldo -= saldo * commission
                        current_position = 'short'
                        trade_count += 1
                elif current_position == 'short':
                    if longCondition[i]:
                        saldo = saldo + (quantidade * (entry_price - close_price[i]))
                        # Aplicar taxa de corretora na venda
                        saldo -= saldo * commission
                        # Inversão para Long
                        entry_price = close_price[i]
                        stopLossLong = entry_price * stoploss_normal_long
                        takeProfitLong = entry_price * stopgain_normal_long
                        quantidade = saldo / entry_price
                        # Aplicar taxa de corretora na compra
                        saldo -= saldo * commission
                        current_position = 'long'
                        trade_count += 1

                # Gerenciamento de Stop Loss e Take Profit
                if current_position == 'long':
                    if low_price[i] <= stopLossLong:
                        saldo = quantidade * stopLossLong
                        position_open = False
                        current_position = None
                    elif high_price[i] >= takeProfitLong:
                        saldo = quantidade * takeProfitLong
                        position_open = False
                        current_position = None
                elif current_position == 'short':
                    if high_price[i] >= stopLossShort:
                        saldo = saldo - (quantidade * (stopLossShort - entry_price))
                        position_open = False
                        current_position = None
                    elif low_price[i] <= takeProfitShort:
                        saldo = saldo + (quantidade * (entry_price - takeProfitShort))
                        position_open = False
                        current_position = None

    # Fechar posição aberta no final do backtest
    if position_open:
        if current_position == 'long':
            saldo = quantidade * close_price.iloc[-1]
        elif current_position == 'short':
            saldo = saldo + (quantidade * (entry_price - close_price.iloc[-1]))
    
    return saldo, trade_count

# Função para gerar listas de parâmetros com base em [início, fim, passo]
def generate_parameter_list(start, end, step, decimals=0):
    if isinstance(start, float) or isinstance(end, float) or isinstance(step, float):
        num = int(np.floor((end - start) / step)) + 1
        return [round(start + step * i, decimals) for i in range(num)]
    else:
        return list(range(start, end + 1, step))

# Função para realizar Random Search
def random_search(df, param_grid, n_iterations, seed=None):
    if seed is not None:
        random.seed(seed)
        np.random.seed(seed)
    
    keys = list(param_grid.keys())
    results = []
    
    print(f"Iniciando Random Search com {n_iterations} iterações...")
    
    for _ in tqdm(range(n_iterations), desc="Random Search"):
        # Selecionar aleatoriamente um valor para cada parâmetro
        params = {key: random.choice(values) for key, values in param_grid.items()}
        
        # Criar uma tupla de parâmetros na ordem esperada pela função backtest
        param_tuple = (
            params['emaShortLength'],
            params['emaLongLength'],
            params['rsiLength'],
            params['macdShort'],
            params['macdLong'],
            params['macdSignal'],
            params['adxLength'],
            params['adxSmoothing'],
            params['adxThreshold'],
            params['bbLength'],
            params['bbMultiplier'],
            params['lateralThreshold'],
            params['stopgain_lateral_long'],
            params['stoploss_lateral_long'],
            params['stopgain_lateral_short'],
            params['stoploss_lateral_short'],
            params['stopgain_normal_long'],
            params['stoploss_normal_long'],
            params['stopgain_normal_short'],
            params['stoploss_normal_short'],
            params['commission']
        )
        
        # Executar o backtest com os parâmetros selecionados
        saldo_final, trade_count = backtest(param_tuple, df)
        
        # Adicionar os resultados
        result = params.copy()
        result['saldo_final'] = saldo_final
        result['trade_count'] = trade_count
        results.append(result)
    
    # Converter os resultados para um DataFrame
    resultados_df = pd.DataFrame(results)
    return resultados_df

# Carregar o CSV com nome atualizado
df = pd.read_csv('BYBIT_BTCUSDT.P_1h.csv')
# df['time'] = pd.to_datetime(df['time'], unit='s')  # Descomente se necessário

# Definir a grade de parâmetros para a Random Search com intervalos e passos
param_grid_definicao = {
    'emaShortLength': [5, 20, 1],
    'emaLongLength': [20, 100, 1],
    'rsiLength': [7, 21, 1],
    'macdShort': [5, 15, 1],
    'macdLong': [20, 35, 1],
    'macdSignal': [5, 12, 1],
    'adxLength': [7, 28, 1],
    'adxSmoothing': [7, 21, 1],
    'adxThreshold': [20, 40, 1],
    'bbLength': [10, 30, 1],
    'bbMultiplier': [1.5, 3.0, 0.1],
    'lateralThreshold': [0.002, 0.01, 0.001],
    'stopgain_lateral_long': [1.02, 1.10, 0.01],
    'stoploss_lateral_long': [0.90, 0.98, 0.01],
    'stopgain_lateral_short': [0.90, 0.98, 0.01],
    'stoploss_lateral_short': [1.02, 1.10, 0.01],
    'stopgain_normal_long': [1.05, 1.35, 0.01],
    'stoploss_normal_long': [0.80, 0.95, 0.01],
    'stopgain_normal_short': [0.70, 0.95, 0.01],
    'stoploss_normal_short': [1.05, 1.15, 0.01],
    'commission': [0.00032]  # Fixa
}

# Gerar listas de valores para cada parâmetro
param_grid = {
    key: generate_parameter_list(*values) if key != 'commission' else values
    for key, values in param_grid_definicao.items()
}

# Definir o número de iterações para a Random Search
n_iterations = 10000  # Ajuste conforme a capacidade computacional

# Executar a Random Search
resultado_random = random_search(df, param_grid, n_iterations, seed=42)

# Ordenar os resultados pelo saldo final em ordem decrescente
resultado_random_sorted = resultado_random.sort_values(by='saldo_final', ascending=False)

# Exibir os top 5 resultados
print("\nTop 5 Resultados da Random Search:")
print(resultado_random_sorted.head(5))

# Salvar os resultados em um arquivo CSV para análise posterior
resultado_random_sorted.to_csv('random_search_resultados.csv', index=False)


Iniciando Random Search com 100 iterações...


Random Search: 100%|██████████| 100/100 [01:21<00:00,  1.22it/s]


Top 5 Resultados da Random Search:
    emaShortLength  emaLongLength  rsiLength  macdShort  macdLong  macdSignal  \
70               7             85         17          6        26          10   
31              10             59          8         14        20           9   
19              12             54          9          6        32           5   
58               6             62         10          7        26           6   
57              19             50         20         10        23          10   

    adxLength  adxSmoothing  adxThreshold  bbLength  ...  \
70         18            18            40        14  ...   
31         25            17            32        22  ...   
19         22            10            26        24  ...   
58         24            10            38        16  ...   
57         24            21            40        21  ...   

    stoploss_lateral_long  stopgain_lateral_short  stoploss_lateral_short  \
70                    1.0              

In [8]:
from joblib import Parallel, delayed

# Função para realizar uma única iteração do Random Search
def single_iteration(df, param_grid):
    # Selecionar aleatoriamente um valor para cada parâmetro
    params = {key: random.choice(values) for key, values in param_grid.items()}
    
    # Criar uma tupla de parâmetros na ordem esperada pela função backtest
    param_tuple = (
        params['emaShortLength'],
        params['emaLongLength'],
        params['rsiLength'],
        params['macdShort'],
        params['macdLong'],
        params['macdSignal'],
        params['adxLength'],
        params['adxSmoothing'],
        params['adxThreshold'],
        params['bbLength'],
        params['bbMultiplier'],
        params['lateralThreshold'],
        params['stopgain_lateral_long'],
        params['stoploss_lateral_long'],
        params['stopgain_lateral_short'],
        params['stoploss_lateral_short'],
        params['stopgain_normal_long'],
        params['stoploss_normal_long'],
        params['stopgain_normal_short'],
        params['stoploss_normal_short'],
        params['commission']
    )
    
    # Executar o backtest com os parâmetros selecionados
    saldo_final, trade_count = backtest(param_tuple, df)
    
    # Adicionar os resultados
    result = params.copy()
    result['saldo_final'] = saldo_final
    result['trade_count'] = trade_count
    return result

# Função para realizar Random Search com paralelização
def random_search_parallel(df, param_grid, n_iterations, n_jobs=-1, seed=None):
    if seed is not None:
        random.seed(seed)
        np.random.seed(seed)
    
    print(f"Iniciando Random Search com {n_iterations} iterações em paralelo...")
    
    # Executar as iterações em paralelo
    results = Parallel(n_jobs=n_jobs)(
        delayed(single_iteration)(df, param_grid) for _ in range(n_iterations)
    )
    
    # Converter os resultados para um DataFrame
    resultados_df = pd.DataFrame(results)
    return resultados_df

# Executar a Random Search com paralelização
resultado_random_parallel = random_search_parallel(df, param_grid, n_iterations=10000, n_jobs=-1, seed=42)

# Ordenar os resultados pelo saldo final em ordem decrescente
resultado_random_parallel_sorted = resultado_random_parallel.sort_values(by='saldo_final', ascending=False)

# Exibir os top 5 resultados
print("\nTop 5 Resultados da Random Search (Paralelizado):")
print(resultado_random_parallel_sorted.head(5))

# Salvar os resultados em um arquivo CSV para análise posterior
resultado_random_parallel_sorted.to_csv('random_search_resultados_paralelizado.csv', index=False)


NameError: name 'param_grid' is not defined